In [1]:
%cd /home/istalter/plantclef-2024/
from plantclef.utils import get_spark

# https://knowledge.informatica.com/s/article/000196886?language=en_US
# The vectorized reader will run out of memory (8gb) with the default batch size, so
# this is one way of handling the issue. This is likely due to the fact that the data
# column is so damn big, and treated as binary data instead of something like a string.
# We might also be able to avoid this if we don't cache the fields into memory, but this
# this needs to be validated by hand. 
spark = get_spark(**{
    # "spark.sql.parquet.columnarReaderBatchSize": 512,
    "spark.sql.parquet.enableVectorizedReader": False, 
})

size = 'small' # small, medium, large
gcs_parquet_path = "gs://dsgt-clef-plantclef-2024/data/parquet_files/"
input_folder = f"PlantCLEF2024_training_cropped_resized_v2/"

df = spark.read.parquet(gcs_parquet_path+input_folder)
df.printSchema()
# df.show(1, vertical=True, truncate=True)
df.count()

[Errno 2] No such file or directory: '/home/istalter/plantclef-2024/'
/Users/istalter/Desktop/plantclef-2024/notebooks/embedding


/Users/istalter/Library/Python/3.11/lib/python/site-packages/IPython/core/magics/osm.py:393: UserWarning: This is now an optional IPython functionality, using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})


ModuleNotFoundError: No module named 'plantclef'

In [2]:
from pyspark.ml.functions import predict_batch_udf
from pyspark.sql.types import BinaryType, ArrayType, FloatType
import io
from PIL import Image
import torch
import numpy as np

def make_predict_fn():
    """Return PredictBatchFunction"""
    from transformers import AutoImageProcessor, AutoModel
    ### need to update this with the pretrained dino
    processor = AutoImageProcessor.from_pretrained('facebook/dinov2-base')
    model = AutoModel.from_pretrained('facebook/dinov2-base')

    def predict(inputs: np.ndarray) -> np.ndarray:
        # print('inputs:')
        # print(type(inputs))
        # print(inputs.shape)

        images = [Image.open(io.BytesIO(input)) for input in inputs]
        # print('images:')
        # print(type(images))
        # print(images)

        model_inputs = processor(images=images, return_tensors="pt")
        # print('model_inputs:')
        # print(type(model_inputs))

        with torch.no_grad():
            # print('start modeling')
            outputs = model(**model_inputs)
            # print('outputs')
            # print(outputs)
            last_hidden_states = outputs.last_hidden_state
        
        # print('last_hidden_states:')
        # print(type(last_hidden_states))
        # print(last_hidden_states.shape)

        numpy_array = last_hidden_states.numpy()
        # Reshape the array
        new_shape = numpy_array.shape[:-2] + (-1,)
        numpy_array = numpy_array.reshape(new_shape)

        # print('numpy_array:')
        # print(type(numpy_array))
        # print(numpy_array.shape)

        return numpy_array

    return predict
    
# batch prediction UDF
apply_dino_pbudf = predict_batch_udf(
    make_predict_fn = make_predict_fn,
    return_type=ArrayType(FloatType()),
    batch_size=8
)

ModuleNotFoundError: No module named 'pyspark'

In [ ]:
# Apply the UDF to transform images
df_transformed = df.limit(24).withColumn("transformed_image", apply_dino_pbudf(df["data"]))

df_transformed.printSchema()